In [2]:
%%capture
!pip install unsloth
!pip install datasets
!pip install -U huggingface_hub
!pip install wandb

In [3]:
# from datasets import load_dataset
# import json

# dataset = load_dataset("Congliu/Chinese-DeepSeek-R1-Distill-data-110k-SFT", streaming=True)
# filtered_dataset = dataset.filter(lambda x: x['repo_name'] == 'zhihu/zhihu_score9.0-10_clean_v10')

# def save_to_json(dataset, filename):
#     with open(filename, 'w', encoding='utf-8') as f:
#         for x in dataset:
#             json.dump(x, f, ensure_ascii=False)
#             f.write('\n')

# save_to_json(filtered_dataset['train'], 'zhihu_data.json')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.03k [00:00<?, ?B/s]

In [14]:
from datasets import load_dataset
import json

dataset = load_dataset("Congliu/Chinese-DeepSeek-R1-Distill-data-110k-SFT", streaming=True)
filtered_dataset = dataset.filter(lambda x: x['repo_name'] == 'xhs/xhs')

def save_to_json(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for x in dataset:
            json.dump(x, f, ensure_ascii=False)
            f.write('\n')

save_to_json(filtered_dataset['train'], 'xhs_data.json')

In [4]:
from huggingface_hub import login
hf_token = "xxx"
login(hf_token)

In [ ]:
import wandb

wb_token = "xxx"
wandb.login(key=wb_token)
run = wandb.init(
    project='fine_tune_deepseek',
    job_type="training",
    anonymous="allow"
)

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

# apply flash attention
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name="unsloth/DeepSeek-R1-Distill-Qwen-14B",
#     max_seq_length=2048,
#     dtype="bf16",
#     load_in_4bit=True,
#     token=hf_token,
#     attn_implementation="flash_attention_2",
# )

In [15]:
train_prompt_style = """以下是一项任务说明，并附带了更详细的背景信息。
请撰写一个满足完成请求的回复。
在回答之前，请仔细考虑问题，并创建一个逐步的思考链，以确保逻辑和准确的回答。

### Instruction:
你是一个资深的小红书文案写作专家，请基于用户输入的问题完成写作。
### Question:
{}
### Response:
<think>
{}
</think>
{}"""
EOS_TOKEN = tokenizer.eos_token

In [9]:
import re
from datasets import load_dataset
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction_text, output_text in zip(instructions, outputs):
        match = re.search(r"<think>(.*?)</think>", output_text, re.DOTALL)
        cot = match.group(1).strip() if match else ""
        output = output_text.replace(match.group(0), "").strip() if match else output_text.strip()
        text = train_prompt_style.format(instruction_text, cot, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2534 [00:00<?, ? examples/s]

In [16]:
dataset = load_dataset("json", data_files="xhs_data.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1507 [00:00<?, ? examples/s]

In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir="outputs",
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=4,
#     warmup_steps=100,
#     max_steps=2000,  #
#     learning_rate=5e-5,
#     bf16=True,
#     logging_steps=10,
#     optim="paged_adamw_32bit",
#     weight_decay=0.05,
#     lr_scheduler_type="cosine_with_restarts",
#     seed=3407,
#     report_to="wandb",
# )

In [20]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=2000,
        learning_rate=5e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="paged_adamw_32bit",
        weight_decay=0.05,
        lr_scheduler_type="cosine_with_restarts",
        seed=3407,
        output_dir="outputs",
        report_to="wandb",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r outputs /content/drive/MyDrive/